# Day 21

https://adventofcode.com/2021/day/21

Part 1 is easy. For part 2, the possible number of games is too large to model each one.

The game state before each round is defined by four numbers - the current square and score so far for each player. Notice that if player 1 is on 20 points then she is guaranteed to win in the next round whatever happens. So start with player 1 on 20 points and work backwards through every game state, recording the number of wins for each player from each state. From earlier game states, either one player will win, or after the round the game will be in one of the states already looked at, so we can look up the number of wins. Work all the way back to zero points for each player and then look up the answer.

One problem with my approach is that I always played a full round (player 1 then player 2), but actually if player 1 wins then player 2 doesn't get to play. The impact of this is that player 1's wins are counted 27 times. To solve this I also counted player 2's wins 27 times, then divided the answer by 27 at the end.

In [1]:
import numpy as np

In [2]:
class Die:
    
    def __init__(self):
        self.last_value = 0
        self.times_rolled = 0
        
    def roll(self):
        res = sum(range(self.last_value + 1, self.last_value + 4))
        self.last_value = self.last_value + 3
        if self.last_value > 100:
            self.last_value = self.last_value - 100
        self.times_rolled = self.times_rolled + 3
        return res
    

In [3]:
starting_positions = [8, 3]

test_positions = [4, 8]

def play(start_positions):
    """Simulate determininstic game up to a score of 1000. Return the product of the 
    losing score and the number of rolls."""
    positions = start_positions
    scores = [0, 0]
    die = Die()
    game_ends = False
    while not game_ends:
        for p in range(2):
            new_roll = die.roll()
            new_position = positions[p] + new_roll
            new_position = new_position % 10
            if new_position == 0:
                new_position = 10
            positions[p] = new_position
            scores[p] = scores[p] + new_position
            if scores[p] >= 1000:
                game_ends = True
                break
            
    losing_score = min(scores)
    return losing_score * die.times_rolled

play(starting_positions)

412344

In [4]:
"""All possible permutations of six dice rolls."""

from itertools import product 

dice_faces = range(1, 4)
                        
all_dice_rolls = np.array(
    list(product(dice_faces, dice_faces, dice_faces, dice_faces, dice_faces, dice_faces)),
    dtype = "int64")

dice_rolls = np.transpose(np.vstack((np.sum(all_dice_rolls[:, :3], axis = 1),
    np.sum(all_dice_rolls[:, 3:], axis = 1))))

In [5]:
def evaluate_position(t, wins_losses):
    """Return the number of wins and losses from a given position t, using the 
    number of wins and losses from all possible future positions of the game."""
    if t[2] >= 21:
        return np.array([1, 0], dtype = np.int64)
    elif t[3] >= 21:
        return np.array([0, 27], dtype = np.int64)
    elif t in wins_losses:
        return wins_losses[t]
    else:
        raise Exception("unevaluated position: " + str(t))

def play_round(position, wins_losses):
    """Play a round of the game with every possible permutation of the 6 dice rolls."""
    squares = np.array(position[0:2], dtype = "int64")
    points = np.array(position[2:4], dtype = "int64")
    new_squares = squares + dice_rolls
    
    new_squares[new_squares > 10] = new_squares[new_squares > 10] - 10
    new_points = points + new_squares
    
    new_positions = [tuple(r) for r in np.hstack((new_squares, new_points))]
    res = [evaluate_position(t, wins_losses) for t in new_positions]
    
    return np.sum(res, axis = 0, dtype = np.int64)

In [6]:
"""Work backwards through game positions to find number of wins/losses from each state"""

wins_losses = dict()

for your_points in range(20, -1, -1):
    print(your_points)
    for opponent_points in range(21):
        for your_square in range(1, 11):
            for opponent_square in range(1, 11):
                position = (your_square, opponent_square, your_points, opponent_points)
                wins_losses[position] = play_round(position, wins_losses)
            


20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [7]:
np.max((wins_losses[(8, 3, 0, 0)]/27).astype(np.int64))

214924284932572